# TF-IDF

https://en.wikipedia.org/wiki/Tf%E2%80%93idf

TFIDF (Term Frequency - Inverse Document Frequency) is a statistical method used to quantify the importance of words within a given text, compared to a background corpus.

How does this work?

https://triton.ml/blog/tf-idf-from-scratch

## TF-IDF from scratch

First, lets read in the complete set of publication data.

In [2]:
import config
import csv
import pandas as pd

#read in the preprocessed publication data
file = open(config.demoPubmedFile)
reader = csv.reader(file,delimiter='\t')
pubData=[]
for row in reader:
    text=row[2]+' '+row[3]
    pubData.append(text.lower().split())
    
#Removes header
pubData = pubData[1:]
print(len(pubData),'publications')


11160 publications


#### Computing a TF Map 

(Modified from https://triton.ml/blog/tf-idf-from-scratch)

**TF(term) = # of times the term appears in document / total # of terms in document** 

Now that our data is usable, we’d like to start computing the TF and the IDF. Computing the tf of a word in a publication requires us to calculate the number of words in a publication, and the number of times each word appears in the publication. We can store each (word, word count pair) in a dictionary. The keys of the dictionary are then just the unique terms in the publication. The following function takes in a publication and outputs a tf dictionary for that publication.

In [3]:
def computePublicationTFDict(publication):
    """ Returns a tf dictionary for each publication whose keys are all 
    the unique words in the publication and whose values are their 
    corresponding tf.
    """
    #Counts the number of times the word appears in publication
    publicationTFDict = {}
    for word in publication:
        if word in publicationTFDict:
            publicationTFDict[word] += 1
        else:
            publicationTFDict[word] = 1
    #Computes tf for each word           
    for word in publicationTFDict:
        publicationTFDict[word] = publicationTFDict[word] / len(publication)
    return publicationTFDict

In [18]:
#run for each list
tfDict={}
for d in range(0,len(pubData)):
    tfDict[d]=computePublicationTFDict(pubData[d])

#create df for first publication
resDic=[]
for t in tfDict[0]:
    resDic.append({'term':t,'tf':tfDict[0][t]})
df=pd.DataFrame.from_dict(resDic)    
print(df.head())

         term    tf-idf
0  sixty-five  0.004184
1      common  0.004184
2     genetic  0.008368
3    variants  0.004184
4         and  0.016736


#### Computing an IDF Map

(Modified from https://triton.ml/blog/tf-idf-from-scratch)

**IDF(term) = log(total # of documents / # of documents with term in it)** 

Computing the idf of a word requires us to compute the total number of documents and the number of documents that contains the word. In our case we can calculate the total number of documents with len(data), the number of publications. For each publication, we increment the document count for each unique word. We can use the keys of the dictionaries that we calculated in the TF step to get the unique set of words. The resulting IDF dictionary’s keys will be the set of all unique words across every document.

In [19]:
def computeCountDict():
    """ Returns a dictionary whose keys are all the unique words in
    the dataset and whose values count the number of reviews in which
    the word appears.
    """
    countDict = {}
    # Run through each publications's tf dictionary and increment countDict's (word, doc) pair
    for review in tfDict:
        for word in tfDict[review]:
            if word in countDict:
                countDict[word] += 1
            else:
                countDict[word] = 1
    return countDict

#Stores the publication count dictionary
countDict = computeCountDict()
testWord='genetic'
print(testWord,countDict[testWord])

genetic 1001


Finally, we can compute an idfDict, using countDict and some math, and store it.

In [20]:
import math

def computeIDFDict():
    """ Returns a dictionary whose keys are all the unique words in the
    dataset and whose values are their corresponding idf.
    """
    idfDict = {}
    for word in countDict:
        idfDict[word] = math.log(len(pubData) / countDict[word])
    return idfDict
  
#Stores the idf dictionary
idfDict = computeIDFDict()

print(idfDict["genetic"])

2.4113364566200812


In this case there are 11,160 publictions, and the word genetic is mentioned 1,001 times. Therefore the idf = log(11160/1001).

#### Computing the TF-IDF Map 

(Modified from https://triton.ml/blog/tf-idf-from-scratch)

**TF-IDF(term) = TF(term) * IDF(term)**

The last step is to compute the TF-IDF. We use our existing tf dictionaries and simply multiply each value by the idf. We can use the idf keys since they contain every unique word.

In [22]:
def computeReviewTFIDFDict(reviewTFDict):
    """ Returns a dictionary whose keys are all the unique words in the
    review and whose values are their corresponding tfidf.
    """
    reviewTFIDFDict = {}
    #For each word in the publication, we multiply its tf and its idf.
    for word in tfDict[reviewTFDict]:
        reviewTFIDFDict[word] = tfDict[reviewTFDict][word] * idfDict[word]
    return reviewTFIDFDict

#Stores the TF-IDF dictionaries
tfidfDict = [computeReviewTFIDFDict(review) for review in tfDict]
#print(tfidfDict[0])

#create df for first publication
resDic=[]
for t in tfDict[0]:
    resDic.append({'term':t,'tf-idf':tfDict[0][t]})
df=pd.DataFrame.from_dict(resDic)    
print(df.head())

         term    tf-idf
0  sixty-five  0.004184
1      common  0.004184
2     genetic  0.008368
3    variants  0.004184
4         and  0.016736


So, using the **genetic** example before, for publication 1, we had a tf value of **'genetic': 0.008368200836820083** and the idf is 2.4113364566200812. Multiply these together, and you get 0.02 (ish) as seen above :)

## TF-IDF using sklearn

The above has been implemented in the python package scikit-learn (sklearn) - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html and can be achieved in just a few lines.

First let's use some functions to load in the data

In [4]:
from scripts.common_functions import orcid_to_pubmed

#create dictionary of orcid to publication text
orcidToPubmed = orcid_to_pubmed()

print('Reading corpus')
token_dict = {}
for orcid in orcidToPubmed:
    token_dict[orcid] = orcidToPubmed[orcid].lower()
print('Done')

orcid_to_pubmed
load_orcid
load_pubmed
Reading corpus
Done


Now let's take a look at the sklean TfidfVectorizer (Equivalent to [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) followed by [TfidfTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer)).

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
    
#setup sklean TfidfVectorizer including bigrams and trigrams
print('Setup')
tfidf = TfidfVectorizer(stop_words='english',ngram_range=(1,3))


#fit_transform creates the tf-idf model and returns term-document frequency matrix
print('Transform')
%time tfs = tfidf.fit_transform(token_dict.values())

#get similarity matrix for all people
#https://stackoverflow.com/questions/8897593/how-to-compute-the-similarity-between-two-text-documents
print('Create matrix')
matrix=(tfs * tfs.T).A

#store this and the dictionary for access in other notebooks
print('Store')
%store matrix
%store token_dict
%store tfs
%store tfidf

print('Done')

Setup
Transform
CPU times: user 14.1 s, sys: 500 ms, total: 14.6 s
Wall time: 12.7 s
Create matrix
Store
Stored 'matrix' (ndarray)
Stored 'token_dict' (dict)
Stored 'tfs' (csr_matrix)
Stored 'tfidf' (TfidfVectorizer)
Done


And to test a document:

In [9]:
def tfidf_doc(tfidf='',text=''):
    text=text.lower()
    #transform function transforms a document to document-term matrix
    response = tfidf.transform([text])

    #get the feature name from the model
    feature_names = tfidf.get_feature_names()
    res={}
    for col in response.nonzero()[1]:
        res[feature_names[col]]=response[0, col]
        #reverse sort the results
        sorted_res = sorted(res.items(), key=lambda kv: kv[1], reverse=True)
    return sorted_res

#create a test document using the first publication 
testText=",".join(pubData[0])
sorted_res=tfidf_doc(tfidf=tfidf,text=testText)
for s in sorted_res[:10]:
    print(s)

#get the genetic score for comparison to above
for s in sorted_res:
    if s[0] == 'genetic':
        print(s)


('t2d', 0.2771374468347099)
('gene score', 0.18732077434507402)
('95 ci', 0.1680563752219566)
('ci', 0.16519742224518033)
('95', 0.14483197609614537)
('score', 0.14159779049586887)
('risk model', 0.12883139598887866)
('nri', 0.09366038717253701)
('framingham', 0.09366038717253701)
('27 kg', 0.09064520376269428)
('genetic', 0.03563686144053027)


#### Why the different tf-idf scores to the first time?

The first example treated each publication separately. In the sklearn example above, each person's collection of publications is treated as a single record. This does mean that there are duplicate publications in the model. Combine this with the slight variation of tf-idf in sklearn, the use of bigrams and trigrams, and the stopwords removal in sklearn, and this might explain the difference. 

For example, the top term in sklean is **t2d** (type 2 diabetes). However, in the manual tf-idf, method this is split across three results **(t2d)**, **td.** and **td**



## TF-IDF on our data

We can now identify the key words in each person's publications, by creating a single document of all texts and comapring to the background frequencies. 

For example:

In [10]:
orcidToPubmedID=load_orcid()
pubmedText = load_pubmed()

#get all publications for a specific ORCID
orcidID='0000-0001-7328-4233'
oText=''
for p in orcidToPubmedID[orcidID]:
    if p in pubmedText:
        oText+=(pubmedText[p])
res = tfidf_doc(tfidf=tfidf,text=oText)
for r in res[0:10]:
    print(r)


load_orcid
load_pubmed
('nematode', 0.1399503126881953)
('mammary', 0.1399503126881953)
('ccr5', 0.13745566504259402)
('motu', 0.11465349697937983)
('crpc', 0.10690996169979536)
('cell', 0.09380439488333107)
('cancer', 0.09324539850690329)
('mammary stem', 0.09163711002839603)
('id4', 0.09163711002839603)
('genome', 0.08541729113595346)


We can now easily do this for all ORCID

In [ ]:
o=open(config.tfidfFile,'w')
counter=0

orcidToPubmedID=load_orcid()
pubmedText = load_pubmed()
for orcid in orcidToPubmed:
    #don't really want to do this for all, so just orcid with < 100 publications!
    if len(orcidToPubmedID[orcid])<100:
        counter+=1
        if counter<=5:
            print(counter,orcid)
            oText=''
            for p in orcidToPubmedID[orcid]:
                if p in pubmedText:
                    oText+=(pubmedText[p])
            print(len(oText))
            %time res = tfidf_doc(tfidf=tfidf,text=oText)
            for r in res[0:100]:
                o.write(orcid+'\t'+r[0]+'\t'+str(r[1])+'\n')
o.close()